In [1]:
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
from decimal import Decimal

training_file = "YandexRelPredChallenge.txt"

In [ ]:
# Read data into dataframe
columns = ["SessionID", "TimePassed", "TypeOfAction", "TargetID", "RegionID", 1,2,3,4,5,6,7,8,9,10]
df = pd.read_csv(training_file, sep='\t', header=None, names=columns)

gamma = defaultdict(float)
alpha = defaultdict(lambda: defaultdict(float))

print("Training with EM...")

for i in range(100):

  # Initialise sums and counts
  gamma_count = defaultdict(float)
  alpha_count = defaultdict(lambda: defaultdict(lambda: 2))

  gamma_sum = defaultdict(float)
  alpha_sum = defaultdict(lambda: defaultdict(lambda: 1))

  # Iterate sessions
  grouped = df.groupby("SessionID")
  for session_id, session_df in grouped:

    # Extract session clicks
    session_clicks = session_df[session_df["TypeOfAction"] == "C"]["TargetID"].tolist()

    # Iterate session queries
    for index, row in session_df[session_df["TypeOfAction"] == "Q"].tail(1).iterrows():
      query_id = row["TargetID"]
      for rank in range(1, 11): # from rank 1 to 10
        document_id = row[rank]
        
        # Determine what values should be added to the EM formula sums 
        if document_id in session_clicks:
          gamma_value = alpha_value = 1
        else:
          gamma_value = (gamma[rank] * (1 - alpha[document_id][query_id])) / \
                        (1 - gamma[rank] * alpha[document_id][query_id])
          alpha_value = ((1 - gamma[rank]) * alpha[document_id][query_id]) / \
                        (1 - gamma[rank] * alpha[document_id][query_id])

        gamma_sum[rank] += gamma_value
        alpha_sum[document_id][query_id] += alpha_value

        gamma_count[rank] += 1
        alpha_count[document_id][query_id] += 1    

  # Update variables
  for rank, param in gamma.items():
    gamma[rank] = gamma_sum[rank] / gamma_count[rank]
    
  for document_id, document_params in alpha.items():
    for query_id, param in document_params.items():
      alpha[document_id][query_id] = alpha_sum[document_id][query_id] / alpha_count[document_id][query_id]

  print("Completed EM iteration", i)
  print(gamma, "\n")

Training with EM...
Completed EM iteration 0
defaultdict(<class 'float'>, {4: 0.14227191260561578, 5: 0.1050610224460186, 6: 0.08918665187334641, 7: 0.07749423913971153, 8: 0.07373901169241273, 9: 0.06682597934624904, 10: 0.06861824699155074, 1: 0.6562259964154648, 2: 0.2502347017154562, 3: 0.18050695570538533}) 

Completed EM iteration 1
defaultdict(<class 'float'>, {4: 0.2407974058096939, 5: 0.1815341268526386, 6: 0.15517854411055965, 7: 0.13575702234634737, 8: 0.12935005457425897, 9: 0.11763209772897092, 10: 0.12055004203397085, 1: 0.8344045062681841, 2: 0.3990141463459933, 3: 0.29927487624561383}) 



In [ ]:
model.click_prob(0.1, 1, 0)